In [16]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
path = r'C:\Users\fajar\Documents\Python\Data\All Past Guests - AU NZ.xlsx'
zoho_df = pd.read_excel(path, dtype='string', sheet_name='All Bounce and Unsub')
zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin

zoho_df.fillna('empty', inplace=True)
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Mobile,Birthdate,Year of Birth,...,Lead Sub-Brand,Lead Source,Lead Locations,Lead Source Description,Lead Regions,Lead Status,Email Opt Out,Opt In,Contact Type,Check LSD Rottnest Swim
0,zcrm_3584690000000456086,2019-01-09 11:10:49,empty,Eren,Akpinar,1115422548@44.com,6141065,empty,2000-10-05,2000,...,Other,Past Guests,Karma Kandara,empty,SEAP,Wrong Number,false,false,Email and Phone,Not Exist
1,zcrm_3584690000000456161,2019-01-09 11:10:49,empty,Aarone,Osborne,aaronos623@gmail.com,61434042443,empty,1992-04-23,1992,...,Other,Past Guests,Karma Kandara,empty,SEAP,No Response,false,false,Email and Phone,Not Exist
2,zcrm_3584690000000456275,2019-01-09 11:10:49,empty,Alex,Browmbill,alex.browmbill@gmail.com,61400884154,empty,empty,empty,...,Other,Past Guests,Karma Rottnest,empty,SEAP,New Lead,true,false,Email and Phone,Not Exist
3,zcrm_3584690000000456525,2019-01-09 11:10:52,empty,Amanda,Blackman,amandablakman@bigpond.com,61418557049,empty,empty,empty,...,Other,Past Guests,Karma Rottnest,empty,SEAP,New Lead,true,false,Email and Phone,Not Exist
4,zcrm_3584690000000456609,2019-01-09 11:10:52,empty,Anika,Nel,ankianet@hotmail.com,61499488234,empty,empty,empty,...,Other,Past Guests,Karma Rottnest,empty,SEAP,New Lead,true,false,Email and Phone,Not Exist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18283,zcrm_3584690000158011212,2020-01-09 19:48:59,empty,Rade,Radinovich,rade@streamrecruitment.com.au,61421742504,610421742504,1987-02-15,empty,...,Other,Past Guests,Karma Kandara,empty,SEAP,New Lead,false,true,Email and Phone,Not Exist
18284,zcrm_3584690000163493103,2020-02-06 09:03:52,empty,Tracy,Davidson,tracy.davidson.nz@flightcentre.co.nz,6494452360,empty,empty,empty,...,Other,Past Guests,Karma Kandara,Form Contact Us,SEAP,New Lead,false,false,Email and Phone,Not Exist
18285,zcrm_3584690000310892495,2021-08-27 05:30:44,empty,William,Dobson,william.dobson@cranehire.co.nz,empty,empty,empty,empty,...,Karma Sanctum,Past Guests,Karma Sanctum Soho,empty,empty,empty,true,false,Email Only,Not Exist
18286,zcrm_3584690000433279793,44991.66260416667,empty,Noviarianto,Tju,shissys@yahoo.com,61416568998,empty,empty,empty,...,Other,Past Guests,Karma Royal Jimbaran,Viewpoint,empty,empty,False,True,Email and Phone,Not Exist


In [18]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='booking Start feb 2021')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [19]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')

0            harvey@liblink.co.za
1             _benezi@hotmail.com
2                 0000987@mail.ru
3               0007psc@gmail.com
4              001paras@gmail.com
                   ...           
296284      accounts@strutsol.com
296285         ankitarh@gmail.com
296286    jackdoyle88@hotmail.com
296287      maxdoyle.ei@gmail.com
296288    mtunwind@bigpond.net.au
Name: Email, Length: 296289, dtype: string

In [20]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup.csv', index=False)
find_member_booking.to_excel(r'C:\Users\fajar\Documents\Python\Data\booking-member-dedup(all bounced and unsub).xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [21]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [ ]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [ ]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))